# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

## Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3239540 artículos de Wikipedia repartidos en 3223 archivos. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

      ID <number>
      1234 4522 23 4 4567
      456 21 9890 123 7 0

* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
 
      palabra #ocurrencias
    
  El ID de cada palabra es el número de línea en la cual ocurre.

In [116]:
import pandas as pd
import numpy as np
import time
import gensim
from gensim.models import Word2Vec, KeyedVectors
from math import floor

from corpus import create_coarseNE_df
from corpus import spread_artID

### Cargamos los datos

In [117]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None, 
                           names=['word', 'frequency'], keep_default_na=False) 
                        # con esto evito que el parser trate como nan value a algunas palabras.

In [118]:
print(word_mapping.shape)
word_mapping.head()

(6931358, 2)


,word,frequency
0,the,73411953
1,",",68044881
2,.,54241850
3,of,40550466
4,and,34602894


In [119]:
len(word_mapping['word'].unique())

6931358

### Cargamos los artículos del Documento "0"

In [120]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [121]:
doc_0.shape

(130209, 2)

In [122]:
doc_0.head()

,sentence,art_ID
0,NaN,431388.0
1,650 5590 753942 12 189 243 1 2039 47 257 682 1...,NaN
2,34 23207 1523 4 57193 15 10777 14353 4 157019 ...,NaN
3,418 0 1858 101 0 1322 1 5590 18 860 729 14 92 ...,NaN
4,650 5590 1523 8 1136 15 11301 575 1156 1 2746 ...,NaN


Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [123]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [124]:
doc_0 = doc_0.dropna()

In [125]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

El documento contiene 128395 oraciones.
El documento contiene 1814 artículos


Hacemos que cada sentencia sea una lista de palabras codificadas y casteamos a int

In [126]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: list(map(int, x.split(' '))))

In [127]:
doc_0.head()

,sentence,art_ID
1,"[650, 5590, 753942, 12, 189, 243, 1, 2039, 47,...",431388.0
2,"[34, 23207, 1523, 4, 57193, 15, 10777, 14353, ...",431388.0
3,"[418, 0, 1858, 101, 0, 1322, 1, 5590, 18, 860,...",431388.0
4,"[650, 5590, 1523, 8, 1136, 15, 11301, 575, 115...",431388.0
5,"[152, 642, 16, 8143, 23122, 20, 0, 662955, 16,...",431388.0


Nos quedamos con un artículo

In [128]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

,sentence,art_ID
2756,"[63903, 24730, 9, 7, 3035, 4104, 7584, 1, 355,...",1000.0
2757,"[3654, 16, 2502, 39413, 1, 24730, 9, 44, 3, 99...",1000.0
2758,"[24730, 35, 49, 3521, 15, 575, 1, 15, 2440, 1,...",1000.0
2759,"[152, 112, 8, 2037, 20, 52, 54, 3035, 17572, 3...",1000.0
2760,"[64, 62, 5787, 1132, 15, 0, 144, 24730, 1312, ...",1000.0


Reconstruimos la primera oración del artículo

In [129]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[idx, 'word']
        dec_sentence.append(mapped_w)
    return dec_sentence 

In [130]:
# Escribimos article.columns.get_loc('sentence') para evitar hardcodear el índice correspondiente
# a la columna 'sentence' que en este caso es 0
sentence = article.iloc[0, article.columns.get_loc('sentence')]

In [131]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [132]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

demora: 12.474568605422974


In [133]:
print('Cantidad de word embeddings: {}'.format(len(w2v_model.vectors)))

Cantidad de word embeddings: 3000000


In [134]:
print('Dimensionalidad de los vectores: {}'.format(w2v_model.vector_size))

Dimensionalidad de los vectores: 300


## Exploremos distintas combinaciones de vectores de palabras

### Concatenación

Este método consiste en concatenar los vectores de palabras que rodean una palabra objetivo en un vector más grande, que tiene un tamaño igual a las dimensiones agregadas de todos las proyecciones (embeddings) individuales.

- $w_{ij}$ = peso asociado con la i-ésima dimensión del vector de la j-ésima palabra en la oración. NOTA: con los vectores de palabras de una oración se forma una matriz $w^{\space D\space x\space L}$ donde $L$ es la cantidad de palabras de esa oración.
- $D$ = dimensionalidad de los word vectors originales. Por ejemplo, al usar el modelo word2vec de Google se tiene $D$ = 300.
- $W$ = tamaño de ventana que se define como el número de palabras en un solo lado.

Nos interesa representar el contexto de la I-ésima palabra de la oración. 

La i-ésima dimensión del vector de concatenación, que tiene un tamaño de $2 W D$, se calcula de la siguiente manera:

$$ e_{i} =\begin{cases} 
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space \left\lfloor{\frac{i}{D}}\right\rfloor} & \left\lfloor{\frac{i}{D}}\right\rfloor < W \\
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space 1\space  +\space\left\lfloor{\frac{i}{D}}\right\rfloor} & c.c.
   \end{cases}$$

In [135]:
def concat_vectors(sentence, D, W, model):
    '''
    sentence: list of strings
    D: dimensionality of word vectors
    W: window
    model: word2vec model
    
    return vectors: L x 2WD matrix, where L is the length of the sentence.
    i.e. each row contains a word vector after apply concatenation strategy.
    '''
    L = len(sentence)
    shape = (D, L)    
    sen_matrix = np.zeros(shape)
    vectors = np.zeros((L, 2*W*D))

    # Fill the matrix corresponding to the sentence
    for idx, word in enumerate(sentence):
        try:
            vector = model.get_vector(word)
        except KeyError: # the word is not in the vocabulary
            vector = np.zeros((D))
            
        sen_matrix[:,idx] = vector
    
    # I: index of the target word
    for I in range(0, L):      
        
        concat_vec = np.empty(0)
        
        # Padding with vector of zeros on the left
        if I - W < 0:
            concat_vec = np.append(concat_vec, np.zeros(abs(I-W) * D))

        # Concat vectors from the sentence
        for i in range(I - W, I + W + 1):             
            if i >= 0 and i != I and i < L:
                concat_vec = np.append(concat_vec, sen_matrix[:, i])
            
        # Padding with vector of zeros on the right
        if I + W >= L:
            concat_vec = np.append(concat_vec, np.zeros(abs(I+W+1-L) * D))

        vectors[I,:] = concat_vec
        
    return vectors

### Promedio

Como su nombre indica, se calcula el centroide de los embeddings de todas las palabras circundantes. La fórmula divide cada dimensión en $2W$ ya que el número de palabras del contexto es dos veces el tamaño de la ventana:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} \frac{w_{ij}}{2W}$$

In [136]:
def mean_vectors(sentence, D, W, model):
    '''
    sentence: list of strings
    D: dimensionality of word vectors
    W: window
    model: word2vec model
    
    return vectors: L x D matrix, where L is the length of the sentence.
    i.e. each row contains a word vector after apply mean strategy.
    '''
    L = len(sentence)
    shape = (D, L)    
    sen_matrix = np.zeros(shape)
    vectors = np.zeros((L, D))
    
    # Fill the matrix corresponding to the sentence
    for idx, word in enumerate(sentence):
        try:
            vector = model.get_vector(word)
        except KeyError: # the word is not in the vocabulary
            vector = np.zeros((D))
            
        sen_matrix[:,idx] = vector
        
    for I in range(L):
        vectors[I,:] = np.mean([sen_matrix[:,j] for j in range(I - W, I + W + 1) 
                                if j >= 0 and j != I and j < L], axis=0)
        
    return vectors

### Decaimiento fraccional

Una tercera estrategia para construir un vector de carácteristicas en base a los embeddings de palabras contextuales está inspirada en la forma en que Word2vec combina las palabras en el contexto. Aquí, se supone que la importancia de una palabra para nuestra representación es inversamente proporcional a su distancia respecto a la palabra objetivo.

Por lo tanto, las palabras contextuales se ponderan en función de su distancia de la palabra objetivo:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} *\frac{W - \lvert I-j\rvert}{W}$$


In [137]:
def fractional_decay(sentence, D, W, model):
    '''
    sentence: list of strings
    D: dimensionality of word vectors
    W: window
    model: word2vec model
    
    return vectors: L x D matrix, where L is the length of the sentence.
    i.e. each row contains a word vector after apply fractional decay strategy.
    '''
    L = len(sentence)
    shape = (D, L)
    sen_matrix = np.zeros(shape)
    vectors = np.zeros((L, D))
    
    # Fill the matrix corresponding to the sentence
    for idx, word in enumerate(sentence):
        try:
            vector = model.get_vector(word)
        except KeyError: # the word is not in the vocabulary
            vector = np.zeros((D))
            
        sen_matrix[:,idx] = vector
        
    for I in range(L):
        vectors[I,:] = np.sum([sen_matrix[:,j] * ((W - abs(I-j)) / W) 
                               for j in range(I - W, I + W + 1) 
                                if j >= 0 and j != I and j < L], axis=0)
        
    return vectors

### Decaimiento exponencial

Funciona de manera similar al decaimiento fraccional, que le da más importancia al contexto cercano, pero en este caso la ponderación se realiza exponencialmente:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} * (1 - \alpha)^{\lvert \space I\space-\space j\space\rvert\space-\space1}$$

donde $\alpha = 1 - 0.1^{(W-1)^{-1}}$ es el parámetro de decaimiento. Elegimos el parámetro de tal manera que las palabras inmediatas que rodean a la palabra objetivo contribuyen 10 veces más que las últimas palabras en ambos lados de la ventana.

In [167]:
def exponential_decay(sentence, D, W, model):
    '''
    sentence: list of strings
    D: dimensionality of word vectors
    W: window
    model: word2vec model
    
    return vectors: L x D matrix, where L is the length of the sentence.
    i.e. each row contains a word vector after apply exponential decay strategy.
    '''
    L = len(sentence)
    shape = (D, L)
    sen_matrix = np.zeros(shape)
    vectors = np.zeros((L, D))
    
    # Fill the matrix corresponding to the sentence
    for idx, word in enumerate(sentence):
        try:
            vector = model.get_vector(word)
        except KeyError: # the word is not in the vocabulary
            vector = np.zeros((D))
            
        sen_matrix[:,idx] = vector
    
    # Decay parameter alpha: We choose the parameter in such a way that the immediate words 
    # that surround the target word contribute 10 times more than the last words 
    # on both sides of the window.
    alpha = 1 - (0.1)**((W-1)**(-1))
    
    for I in range(L):
        vectors[I,:] = np.sum([sen_matrix[:,j] * ((1-alpha)**(abs(I-j)-1)) 
                               for j in range(I - W, I + W + 1) 
                                if j >= 0 and j != I and j < L], axis=0)
        
    return vectors

TODO: cambiar redacción y fórmulas de las distintas estrategias si se decide incluir al vector de la palabra objetivo en las operaciones que generan el nuevo vector.

Agregar también que para el caso de concatenación se va a realizar padding de 0's a izquierda o derecha cuando corresponda.

## Exploremos CoarseNE.tar.bz2

Contiene menciones anotadas automáticamente con etiquetas de entidades nombradas (PER, LOC, ORG y MISC).

El formato es:

    ID artID
    sentIdx begin end entityType
    
donde entityType[0] = PER | entityType[1] = LOC | entityType[2] = ORG | entityType[3] = MISC

In [139]:
coarseNE_0 = pd.read_csv('./corpus_WiNER/CoarseNE/0', sep='ID', engine='python', header=None, names=['named-entity', 'art_ID'])

In [140]:
print(coarseNE_0.shape)
coarseNE_0.head()

(259470, 2)


,named-entity,art_ID
0,NaN,1000.0
1,0\t0\t2\t0,NaN
2,0\t5\t6\t1,NaN
3,0\t10\t12\t0,NaN
4,1\t2\t4\t0,NaN


Aplicamos el mismo truco que utilizamos en los documentos para propagar los art_ID según corresponda

In [141]:
coarseNE_0 = spread_artID(coarseNE_0)
coarseNE_0 = coarseNE_0.dropna()

In [142]:
print('coarseNE_0 contiene {} entidades.'.format(coarseNE_0.shape[0]))
print('coarseNE_0 contiene {} artículos'.format(len(coarseNE_0['art_ID'].unique())))

coarseNE_0 contiene 257656 entidades.
coarseNE_0 contiene 1810 artículos


Creamos nuevas columnas con la info de la columna named-entity para mejor manipulación

In [143]:
def get_tag(x):
    tags = ['PER', 'LOC', 'ORG', 'MISC']
    return tags[x]

In [144]:
coarseNE_0['named-entity'] = coarseNE_0['named-entity'].map(lambda x: x.split('\t'))
coarseNE_0['senIdx'] = coarseNE_0['named-entity'].map(lambda x: int(x[0]))
coarseNE_0['begin'] = coarseNE_0['named-entity'].map(lambda x: int(x[1]))
coarseNE_0['end'] = coarseNE_0['named-entity'].map(lambda x: int(x[2]))
coarseNE_0['entityType'] = coarseNE_0['named-entity'].map(lambda x: get_tag(int(x[3])))
coarseNE_0 = coarseNE_0.drop(columns='named-entity')

In [145]:
coarseNE_0.head()

,art_ID,senIdx,begin,end,entityType
1,1000.0,0,0,2,PER
2,1000.0,0,5,6,LOC
3,1000.0,0,10,12,PER
4,1000.0,1,2,4,PER
5,1000.0,1,5,6,PER


No todos los artículos que ocurren en Documents/0 se encuentran en CoarseNE/0

Notar que esto sucede porque esos artículos no contienen entidades nombradas.

In [146]:
print('Artículos que están presentes en Documents/0 pero no en CoarseNE/0: {}'
      .format(list(set(doc_0.art_ID.unique()) - set(coarseNE_0.art_ID.unique()))))

Artículos que están presentes en Documents/0 pero no en CoarseNE/0: [431177.0, 432375.0, 432318.0, 10263.0]


In [147]:
article = doc_0[doc_0.art_ID == 431177]
for sentence in article.sentence.values:
    dec_sentence = sentence_decoder(sentence, word_mapping)
    print(' '.join(dec_sentence))

In economics , economics of location is a strategy used by firms in a monopolistic competition environment .
Unlike a product differentiation strategy , where firms make their products different in order to attract customers , the economics of location strategy causes firms to produce similar or identical products .


### Veamos como están anotadas las entidades nombradas de una oración en particular 

In [149]:
article = doc_0[doc_0.art_ID == 1000]
sentence = article.iloc[0, article.columns.get_loc('sentence')]
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

In [150]:
art_entities = coarseNE_0[coarseNE_0.art_ID == 1000]
entities_sen_0 = art_entities[art_entities.senIdx == 0]
entities_sen_0

,art_ID,senIdx,begin,end,entityType
1,1000.0,0,0,2,PER
2,1000.0,0,5,6,LOC
3,1000.0,0,10,12,PER


In [151]:
for idx, row in entities_sen_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Hercule Poirot : PER
Belgian : LOC
Agatha Christie : PER


In [152]:
doc77_df = pd.read_pickle('./corpus_WiNER/docs_df/doc_77')
doc77_df.head()

,sentence,art_ID
1,"[Belton, House, is, a, Grade, I, listed, count...",145492.0
2,"[The, mansion, is, surrounded, by, formal, gar...",145492.0
3,"[Belton, has, been, described, as, a, compilat...",145492.0
4,"[The, house, has, also, been, described, as, t...",145492.0
5,"[Only, Brympton, d'Evercy, has, been, similarl...",145492.0


In [153]:
article = doc77_df[doc77_df.art_ID == 145492]
article = article.reset_index(drop=True)
article['sen_length'] = article['sentence'].map(lambda x: len(x))
article.head()

,sentence,art_ID,sen_length
0,"[Belton, House, is, a, Grade, I, listed, count...",145492.0,18
1,"[The, mansion, is, surrounded, by, formal, gar...",145492.0,21
2,"[Belton, has, been, described, as, a, compilat...",145492.0,32
3,"[The, house, has, also, been, described, as, t...",145492.0,45
4,"[Only, Brympton, d'Evercy, has, been, similarl...",145492.0,14


In [154]:
dec_sentence = doc77_df.iloc[0, doc77_df.columns.get_loc('sentence')]

In [155]:
coarseNE_77 = pd.read_pickle('./corpus_WiNER/coarseNE_df/coarseNE_77')

In [156]:
art_entities = coarseNE_77[coarseNE_77.art_ID == 145492]
sen_entities_0 = art_entities[art_entities.senIdx == 0]
sen_entities_0
# art_entities.head(20)

,art_ID,senIdx,begin,end,entityType
42515,145492.0,0,0,2,LOC
42516,145492.0,0,10,11,LOC
42517,145492.0,0,12,13,LOC
42518,145492.0,0,14,15,LOC
42519,145492.0,0,16,17,LOC


In [157]:
for idx, row in sen_entities_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Belton House : LOC
Belton : LOC
Grantham : LOC
Lincolnshire : LOC
England : LOC


In [158]:
def senEntities_toList(sen_entities_df, sen_length):
    entities = []
    i = 0
    for _, row in sen_entities_df.iterrows():
        while i < row['begin']:
            entities.append('O')
            i += 1
        while i < row['end']:
            entities.append(row['entityType'])
            i += 1
    while i < sen_length:
        entities.append('O')
        i += 1
    return entities

In [159]:
print(' '.join(dec_sentence))
print(senEntities_toList(sen_entities_0, 18))

Belton House is a Grade I listed country house in Belton near Grantham , Lincolnshire , England .
['LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O']


In [160]:
def artEntities_toList(article_df, art_entities_df):
    art_entities_list = []
    for idx, row in article_df.iterrows():
        sen_entities_list = []
        # We take the df with the entities of each sentence
        sen_entities_df = art_entities_df[art_entities_df.senIdx == idx]
        sen_length = row['sen_length']
        # An empty dataframe means that the sentence doesn't have any entity
        if sen_entities_df.empty:
            sen_entities_list = ['O' for _ in range(sen_length)]
        else:
            sen_entities_list = senEntities_toList(sen_entities_df, sen_length)
        art_entities_list += sen_entities_list
        
    return art_entities_list

In [187]:
article = doc77_df[doc77_df.art_ID == 145492]
article = article.reset_index(drop=True)
article['sen_length'] = article['sentence'].map(lambda x: len(x))
art_entities = coarseNE_77[coarseNE_77.art_ID == 145492]
lista_entidades = artEntities_toList(article, art_entities)
len(lista_entidades)

4395

In [163]:
article.head()

,sentence,art_ID,sen_length
0,"[Belton, House, is, a, Grade, I, listed, count...",145492.0,18
1,"[The, mansion, is, surrounded, by, formal, gar...",145492.0,21
2,"[Belton, has, been, described, as, a, compilat...",145492.0,32
3,"[The, house, has, also, been, described, as, t...",145492.0,45
4,"[Only, Brympton, d'Evercy, has, been, similarl...",145492.0,14


In [164]:
print(' '.join(article['sentence'][4]))
print(lista_entidades[116:130])

Only Brympton d'Evercy has been similarly lauded as the perfect English country house .
['O', 'LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'MISC', 'MISC', 'MISC', 'O']


In [186]:
article.sen_length.sum()

4395

In [262]:
def getArticleVectors(article_df, D, W, w2v_model):
    word_vectors = np.empty((0, D))
    for _, row in article_df.iterrows():
        word_vectors = np.append(word_vectors, 
                                 exponential_decay(row['sentence'], D, W, w2v_model), axis=0)    
    return word_vectors

In [263]:
w = getArticleVectors(article, 300, 10, w2v_model)

In [274]:
w

array([[-0.12550881, -0.03750116, -0.08987626, ..., -0.16715025,
         0.55664425,  0.31988132],
       [-0.18977061, -0.1797001 ,  0.16132336, ..., -0.15613661,
         0.59610407,  0.28217311],
       [-0.22687655, -0.05309891, -0.28841909, ..., -0.29206622,
         0.69761822,  0.35904226],
       ...,
       [ 0.2430228 , -0.12183283, -0.08213317, ..., -0.17359862,
         0.01749144,  0.20006441],
       [ 0.18298432, -0.08041819, -0.06503881, ..., -0.13692519,
        -0.00808196,  0.13592407],
       [ 0.13366328, -0.06576192, -0.03319333, ..., -0.11072303,
        -0.0097546 ,  0.1125754 ]])

In [271]:
dd = pd.DataFrame(w)

In [275]:
dd['entityType'] = lista_entidades

In [279]:
aa = dd.values

In [280]:
ee = pd.DataFrame(aa)

In [287]:
type(ee.loc[0,300])

str

In [ ]:
def genWordVectors_Entity(strategy, W):
    '''
    Generates a N x (D+1) matrix where N is the total amount of words and 
    D is the dimensionality of each word vector. 
    The values of the last column are the entity type (PER - LOC - ORG - MISC - O).
    
    strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
    W: window size
    
    return wordVectors_Entity matrix
    '''
    w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
    D = w2v_model.vector_size
    wordVectors_Entity = np.empty((0, D))
    

Cantidad de oraciones: 54607542
Cantidad de artículos: 3239540